## Amazon Reviews

Data Source: https://nijianmo.github.io/amazon/index.html

This notebook takes in the zipped reviews and meta data from the Amazon review data of Electronics.

Since the files are large and product characteristics vary, we will extract one popular product's information for further modeling and analysis.

### Data Cleaning

Json to dataframe

In [1]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Electronics_5.json.gz')

In [2]:
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5.0,67,True,"09 18, 1999",AAP7PPBU72QFM,0151004714,{'Format:': ' Hardcover'},D. C. Carrad,This is the best novel I have read in 2 or 3 y...,A star is born,937612800,NaN
1,3.0,5,True,"10 23, 2013",A2E168DTVGE6SV,0151004714,{'Format:': ' Kindle Edition'},Evy,"Pages and pages of introspection, in the style...",A stream of consciousness novel,1382486400,NaN
2,5.0,4,False,"09 2, 2008",A1ER5AYS3FQ9O3,0151004714,{'Format:': ' Paperback'},Kcorn,This is the kind of novel to read when you hav...,I'm a huge fan of the author and this one did ...,1220313600,NaN
3,5.0,13,False,"09 4, 2000",A1T17LMQABMBN5,0151004714,{'Format:': ' Hardcover'},Caf Girl Writes,What gorgeous language! What an incredible wri...,The most beautiful book I have ever read!,968025600,NaN
4,3.0,8,True,"02 4, 2000",A3QHJ0FXK33OBE,0151004714,{'Format:': ' Hardcover'},W. Shane Schmidt,I was taken in by reviews that compared this b...,A dissenting view--In part.,949622400,NaN


In [3]:
df.shape

(6739590, 12)

### Aggregate with Metadata

#### Group reviews data by products

asin: Amazon Standard Identification Number 

In [4]:
tbl = df.groupby('asin').agg({'overall':'count'})
tbl = tbl.sort_values(by = ['overall'], ascending = False)
tbl['asin'] = tbl.index
tbl = tbl.rename_axis("ID")
tbl

,overall,asin
ID,,
B003L1ZYYW,8617,B003L1ZYYW
B0019HL8Q8,8160,B0019HL8Q8
B0019EHU8G,7777,B0019EHU8G
B0015DYMVO,7380,B0015DYMVO
B000VS4HDM,6802,B000VS4HDM
...,...,...
B000YJAWUA,1,B000YJAWUA
B0013MYX8E,1,B0013MYX8E
B000WGS5B8,1,B000WGS5B8


#### Read in metadata and merge with aggregated review data

In [5]:
# read in meta data and inner merge with review data by asin
meta = pd.read_csv('meta.csv')
new = tbl.merge(meta, on = 'asin', how = 'inner')

C:\Users\lliu9\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
new.columns

Index(['overall', 'asin', 'Unnamed: 0', 'category', 'description', 'title',
       'image', 'brand', 'feature', 'rank', 'main_cat', 'date', 'price',
       'also_buy', 'also_view', 'similar_item', 'tech1', 'tech2', 'details',
       'fit'],
      dtype='object')

#### Exploratory playground

In [7]:
def keep_recent(new):
    '''The function returns whether the product is released after 2016.'''
    try:
        if int(new['date'][-4:]) > 2016:
            return 1
        else:
            return 0
    except:
        return 0
    
new['newitem'] = new.apply(keep_recent, axis = 1)

In [8]:
sim_df = new[new['newitem'] == 1][['asin', 'title', 'overall', 'brand']]

In [9]:
sim_df.head(10)

,asin,title,overall,brand
19,B00BP5KOPA,Logitech MK270 Wireless Keyboard and Mouse Com...,4625,Logitech
26,B001TH7GUU,AmazonBasics USB 2.0 Extension Cable - A-Male ...,4503,AmazonBasics
36,B009D79VH4,Transcend USB 3.0 SDHC / SDXC / microSDHC / SD...,3747,Transcend
48,B001XURP8Q,"Sandisk Cruzer 32GB USB 32 GB Flash Drive, Bla...",3166,SanDisk
51,B004OVECU0,Logitech Harmony 650 Infrared All in One Remot...,3132,Logitech
61,B001TH7GSW,AmazonBasics Digital Optical Audio Toslink Cab...,3016,AmazonBasics
75,B00APCMMEK,Transcend 16GB MicroSDHC Class10 UHS-1 Memory ...,2848,Transcend
89,B000WU2LXC,"ARCTIC MX-2 - Thermal Compound Paste, Carbon B...",2586,ARCTIC
92,B000A6PPOK,Microsoft Natural Ergonomic Keyboard 4000,2567,Microsoft
102,B00DSUTX3O,WD Black 750GB Performance Mobile Hard Disk Dr...,2442,Western Digital


##### Explore different brands

In [10]:
# Amazon products
sim_df[sim_df['brand']=='Amazon'].head()

,asin,title,overall,brand


In [11]:
# Apple products
sim_df[sim_df['brand']=='Apple'].head()

,asin,title,overall,brand
2913,B00UGBMRQ8,"Apple MacBook Pro 15"" Core i7 2.8GHz Retina (M...",349,Apple
4039,B0096VDM8G,Apple MacBook Pro 15-Inch Laptop with Retina D...,269,Apple
10007,B00B3Y4U4E,Apple MD862ZM/A Thunderbolt Cable - 0.5 M (NEW...,126,Apple
14257,B0186RZAWQ,Apple Smart Keyboard for iPad Pro 12.9,92,Apple
18190,B01H29JY62,"Apple Magic Keyboard (Wireless, Rechargable) (...",73,Apple


In [12]:
sim_df[sim_df['brand']=='Xiaomi'].head()

,asin,title,overall,brand


In [13]:
sim_df[sim_df['brand']=='Huawei'].head()

,asin,title,overall,brand


In [14]:
sim_df[sim_df['brand']=='Lenovo'].head()

,asin,title,overall,brand
15365,B00G6T8EK2,Original ThinkPad Pro Dock ( 40A10090US ) With...,86,Lenovo
19903,B00C32FWJC,ThinkPad Compact Bluetooth Keyboard with Track...,66,Lenovo
34982,B00009YUPY,Lenovo 06P4069 Optical USB Wheel Mouse,37,Lenovo
34983,B00009YUPY,Lenovo 06P4069 Optical USB Wheel Mouse,37,Lenovo
36251,B009ZN8U14,Lenovo Tiny VESA System Mounting Bracket 0B47374,35,Lenovo


#### Explore different products

In [15]:
sim_df[sim_df['title'].str.contains("camera",na = False)].head()

,asin,title,overall,brand
13174,B001TKRLPC,Olympus Water Ready Adjustable Strap for water...,99,Olympus
45141,B001LZ5HDI,Waterproof Case | Pelican 1060 Micro Case - fo...,27,PELICA
96151,B00NB1KDH0,Ashanks 0.5-3KG Pro Handheld Stabilizer/ Mini ...,10,ASHANKS
149022,B01H4OP7Y8,Pixel Vertax D18 battery grip for Nikon D850 c...,5,PIXEL
166021,B0186EQ1MW,Vsafe home HD security camera NEW,5,Vsafe


In [16]:
sim_df[sim_df['title'].str.contains("MacBook",na = False)].head()

,asin,title,overall,brand
621,B00NH12YN0,USB Type C Cable AoLiPlus 6ft LED USB C Cable ...,958,AOLIPLUS
1630,B00D9UTZWW,"iBenzer MacBook Air 11 Inch Case, Soft Touch H...",523,IBENZER
2913,B00UGBMRQ8,"Apple MacBook Pro 15"" Core i7 2.8GHz Retina (M...",349,Apple
4039,B0096VDM8G,Apple MacBook Pro 15-Inch Laptop with Retina D...,269,Apple
6592,B011A286IO,"MacBook Air 13 Inch Case, Lacdo Laptop Hard Ca...",180,Lacdo


In [17]:
sim_df[sim_df['title'].str.contains("Keyboard", na = False)].head(10)

,asin,title,overall,brand
19,B00BP5KOPA,Logitech MK270 Wireless Keyboard and Mouse Com...,4625,Logitech
92,B000A6PPOK,Microsoft Natural Ergonomic Keyboard 4000,2567,Microsoft
191,B00I0S6SK0,Azio Vision Backlit USB Keyboard with Large Pr...,1805,Azio
501,B00KF9IVKC,iPazzPort 3-color Backlit Wireless Mini Keyboa...,1092,iPazzPort
1630,B00D9UTZWW,"iBenzer MacBook Air 11 Inch Case, Soft Touch H...",523,IBENZER
1640,B00NG8EJOG,"Foldable Bluetooth Keyboard, Jelly Comb B003B ...",521,Jelly Comb
2263,B00E4TOWR0,HP Wired USB Keyboard K1500 (Black) (H3C52AA#ABA),418,HP
2547,B00O1B59AU,"ZAGG Slim Book Ultrathin Case, Hinged with Det...",387,ZAGG
2620,B00LSGKEC4,Redragon K502 Gaming Keyboard RGB LED Backlit ...,378,Redragon
2962,B0194MC7JG,Wireless Keyboard and Mouse - Jelly Comb K025 ...,343,Jelly Comb


In [18]:
new[new['asin']=='B00BP5KOPA'].iloc[0]

overall                                                      4625
asin                                                   B00BP5KOPA
Unnamed: 0                                                 363414
category               ['Electronics', 'Computers & Accessories']
description     ["The stylish Logitech MK270 Wireless Keyboard...
title           Logitech MK270 Wireless Keyboard and Mouse Com...
image           ['https://images-na.ssl-images-amazon.com/imag...
brand                                                    Logitech
feature         ['WORK FOR LONGER WITH LONG BATTERY LIFE Basic...
rank            >#4 in Computers & Accessories (See top 100)>#...
main_cat                                                Computers
date                                                April 1, 2018
price                                                      $19.99
also_buy                                                      NaN
also_view       ['B003VAGXZC', 'B00QXT5T3U', 'B00L1Y11D4', 'B0...
similar_it

In [19]:
df[df['asin']=='B000A6PPOK']

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
376345,5.0,"1,113",True,"10 18, 2005",AILCWT1IIP7ZT,B000A6PPOK,{'Style:': ' Retail'},Charles Chen,"As a software developer, I am literally attach...",Best Overall Keyboard to Date,1129593600,NaN
376346,5.0,16,True,"10 12, 2005",A300T403J8526F,B000A6PPOK,{'Style:': ' Retail'},T. Becker,"I've had a Natural Keyboard Pro for years, and...",Best keyboard I've used,1129075200,NaN
376347,4.0,11,True,"10 5, 2005",AFPGV3IQ9K691,B000A6PPOK,{'Style:': ' Retail'},GDC,"As with most Microsoft products, great hardwar...","Great keyboard, poor software.",1128470400,NaN
376361,5.0,NaN,True,"05 15, 2018",A2FMMCFQ8702DN,B000A6PPOK,{'Style:': ' Business'},Wickenball,Love it.,Five Stars,1526342400,NaN
376362,5.0,NaN,True,"05 12, 2018",A1I8WBJSZIPJE0,B000A6PPOK,{'Style:': ' Business'},B. Stearn,I've had various versions of this keyboard for...,Five Stars,1526083200,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
379488,1.0,11,False,"10 20, 2005",A3EXEZBIKEP5U3,B000A6PPOK,{'Style:': ' Retail'},K. HUANG,"From the pictures it looks like the same, poor...",Real typers beware - 6 key is on the wrong side,1129766400,NaN
6143029,5.0,NaN,True,"05 19, 2018",A3NOZ4M9D1IIR7,B000A6PPOK,{'Style:': ' Business'},john chamberlain,good,Five Stars,1526688000,NaN
6143030,5.0,NaN,True,"05 17, 2018",A36WZAWDM2D487,B000A6PPOK,{'Style:': ' Retail'},Amazon Customer,This is my second time purchasing this keyboar...,The keyboard is dropped on the floor 2 to 3 ti...,1526515200,NaN
6143031,5.0,NaN,True,"05 16, 2018",ALZ9C6MTMF20E,B000A6PPOK,{'Style:': ' Business'},D.J.,My favorite keyboard by far. Only issue is tha...,excellent product,1526428800,NaN


### Extract Sample Data

In [20]:
#pd.set_option('display.max_colwidth', 1)
print(new[new['asin']=='B000A6PPOK']['title'])

92    Microsoft Natural Ergonomic Keyboard 4000
Name: title, dtype: object


In [21]:
new[new['asin']=='B000A6PPOK'].head()

,overall,asin,Unnamed: 0,category,description,title,image,brand,feature,rank,...,date,price,also_buy,also_view,similar_item,tech1,tech2,details,fit,newitem
92,2567,B000A6PPOK,73287,"['Electronics', 'Computers & Accessories', 'Co...",['USB'],Microsoft Natural Ergonomic Keyboard 4000,['https://images-na.ssl-images-amazon.com/imag...,Microsoft,['Split ergonomic design encourages natural ha...,['>#222 in Computers & Accessories > Computer ...,...,"September 27, 2017",NaN,"['B00FPAVUHC', 'B0043T7FXE', 'B001F42MKG', 'B0...","['B075GZVD4T', 'B002MMY4WY', 'B00CYX26BC', 'B0...","[{'asin': '', 'href': '', 'name': 'Microsoft N...",NaN,NaN,NaN,NaN,1


In [22]:
df.columns

Index(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'image'],
      dtype='object')

In [23]:
pdt = df[df['asin']=='B000A6PPOK']
pdt.to_csv('sample.csv')

### Extract Larger Sample Data

In [24]:
sim_df[sim_df['title'].str.contains("Keyboard",na = False)].head()

,asin,title,overall,brand
19,B00BP5KOPA,Logitech MK270 Wireless Keyboard and Mouse Com...,4625,Logitech
92,B000A6PPOK,Microsoft Natural Ergonomic Keyboard 4000,2567,Microsoft
191,B00I0S6SK0,Azio Vision Backlit USB Keyboard with Large Pr...,1805,Azio
501,B00KF9IVKC,iPazzPort 3-color Backlit Wireless Mini Keyboa...,1092,iPazzPort
1630,B00D9UTZWW,"iBenzer MacBook Air 11 Inch Case, Soft Touch H...",523,IBENZER


In [25]:
pdt_large = df[df['asin']=='B00BP5KOPA']
pdt_large.to_csv('sample_large.csv')

### Extract Similar Products 

In [45]:
keyboard_top = sim_df[sim_df['title'].str.contains("Keyboard", na = False)].head(11)
keyboard_top = keyboard_top[keyboard_top['asin'].isin(['B000A6PPOK', 'B00I0S6SK0', 'B00E4TOWR0', 'B00LSGKEC4', 'B0194MC7JG'])]

In [61]:
keyboard_top

,asin,title,overall,brand
92,B000A6PPOK,Microsoft Natural Ergonomic Keyboard 4000,2567,Microsoft
191,B00I0S6SK0,Azio Vision Backlit USB Keyboard with Large Pr...,1805,Azio
2263,B00E4TOWR0,HP Wired USB Keyboard K1500 (Black) (H3C52AA#ABA),418,HP
2620,B00LSGKEC4,Redragon K502 Gaming Keyboard RGB LED Backlit ...,378,Redragon
2962,B0194MC7JG,Wireless Keyboard and Mouse - Jelly Comb K025 ...,343,Jelly Comb


In [46]:
keyboard_top['asin']

92      B000A6PPOK
191     B00I0S6SK0
2263    B00E4TOWR0
2620    B00LSGKEC4
2962    B0194MC7JG
Name: asin, dtype: object

In [52]:
output_meta = new[new['asin'].isin(keyboard_top['asin'])].drop(['newitem', 'Unnamed: 0'], axis=1)
output_meta.rename(columns = {'overall': 'ReviewCounts'})
output_meta.to_csv('output_meta.csv')

In [57]:
output_review = df[df['asin'].isin(keyboard_top['asin'])]
output_review = output_review.merge(meta[['asin', 'title']], on = 'asin', how = 'inner')

In [58]:
output_review.columns

Index(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'image', 'title'],
      dtype='object')

In [59]:
output_review = output_review[['asin', 'title', 'overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'image']]

In [60]:
output_review.to_csv('output_review.csv')